<a href="https://colab.research.google.com/github/abalckpie/professorahn/blob/main/%EB%B0%98%EB%B3%B5%EC%B8%A1%EC%A0%95_%EB%B3%80%EB%9F%89%EB%B6%84%EC%84%9D2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
from statsmodels.stats.anova import AnovaRM
from statsmodels.formula.api import mixedlm
from statsmodels.stats.multicomp import pairwise_tukeyhsd

In [ ]:
csv_test = pd.read_csv('/content/2022_2022년 지능정보사회 이용자 패널 조사_Data(가중치포함).csv')
data = csv_test.copy()
columns_of_interest = ['Age_group'] + [f'Q30_{i}' for i in range(1, 25)]
subset_data = data[columns_of_interest]

In [ ]:
subset_data['생활활용'] = subset_data[[f'Q30_{i}' for i in range(1, 7)]].mean(axis=1)
subset_data['권리보호'] = subset_data[[f'Q30_{i}' for i in range(7, 12)]].mean(axis=1)
subset_data['비판적 이해'] = subset_data[[f'Q30_{i}' for i in range(12, 16)]].mean(axis=1)
subset_data['생산과 공유'] = subset_data[[f'Q30_{i}' for i in range(16, 19)]].mean(axis=1)
subset_data['사회 참여'] = subset_data[[f'Q30_{i}' for i in range(19, 22)]].mean(axis=1)
subset_data['보안'] = subset_data[[f'Q30_{i}' for i in range(22, 25)]].mean(axis=1)

<ipython-input-3-50d5c1a80b22>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_data['생활활용'] = subset_data[[f'Q30_{i}' for i in range(1, 7)]].mean(axis=1)
<ipython-input-3-50d5c1a80b22>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_data['권리보호'] = subset_data[[f'Q30_{i}' for i in range(7, 12)]].mean(axis=1)
<ipython-input-3-50d5c1a80b22>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See 

In [ ]:
long_data = pd.melt(subset_data, id_vars=['Age_group'], value_vars=['생활활용', '권리보호', '비판적 이해', '생산과 공유', '사회 참여', '보안'],
                    var_name='Factor', value_name='Response')
long_data

,Age_group,Factor,Response
0,6,생활활용,3.000000
1,5,생활활용,3.333333
2,5,생활활용,5.000000
3,2,생활활용,2.666667
4,4,생활활용,4.000000
...,...,...,...
32263,1,보안,3.666667
32264,4,보안,3.000000
32265,1,보안,4.000000
32266,4,보안,3.000000


In [ ]:
long_data['Age_group'] = long_data['Age_group'].astype('category')
long_data

,Age_group,Factor,Response
0,6,생활활용,3.000000
1,5,생활활용,3.333333
2,5,생활활용,5.000000
3,2,생활활용,2.666667
4,4,생활활용,4.000000
...,...,...,...
32263,1,보안,3.666667
32264,4,보안,3.000000
32265,1,보안,4.000000
32266,4,보안,3.000000


In [ ]:
agg_data = long_data.groupby(['Age_group', 'Factor']).mean().reset_index()
agg_data

,Age_group,Factor,Response
0,1,권리보호,3.163692
1,1,보안,3.447179
2,1,비판적 이해,3.504615
3,1,사회 참여,3.102564
4,1,생산과 공유,3.421538
5,1,생활활용,3.721538
6,2,권리보호,3.378253
7,2,보안,3.651218
8,2,비판적 이해,3.649955
9,2,사회 참여,3.426619


In [ ]:
aovrm = AnovaRM(agg_data, 'Response', 'Age_group', within=['Factor'])
res1 = aovrm.fit()
print(res1)
aovrm = AnovaRM(agg_data, 'Response', 'Factor', within=['Age_group'])
res2 = aovrm.fit()
print(res2)

               Anova
       F Value Num DF  Den DF Pr > F
------------------------------------
Factor 35.7822 5.0000 25.0000 0.0000

                 Anova
          F Value  Num DF  Den DF Pr > F
----------------------------------------
Age_group 103.4679 5.0000 25.0000 0.0000



In [ ]:
tukey1 = pairwise_tukeyhsd(endog=long_data['Response'], groups=long_data['Factor'], alpha=0.05)
print(tukey1)
tukey2 = pairwise_tukeyhsd(endog=long_data['Response'], groups=long_data['Age_group'], alpha=0.05)
print(tukey2)

Multiple Comparison of Means - Tukey HSD, FWER=0.05 
group1 group2 meandiff p-adj   lower   upper  reject
----------------------------------------------------
  권리보호     보안   0.0702 0.0002  0.0238  0.1166   True
  권리보호 비판적 이해   0.2334    0.0   0.187  0.2798   True
  권리보호  사회 참여   0.0346 0.2727 -0.0118   0.081  False
  권리보호 생산과 공유  -0.0097 0.9912 -0.0561  0.0367  False
  권리보호   생활활용   0.5765    0.0  0.5301  0.6229   True
    보안 비판적 이해   0.1631    0.0  0.1167  0.2095   True
    보안  사회 참여  -0.0356 0.2446  -0.082  0.0108  False
    보안 생산과 공유    -0.08    0.0 -0.1264 -0.0336   True
    보안   생활활용   0.5063    0.0  0.4599  0.5527   True
비판적 이해  사회 참여  -0.1987    0.0 -0.2451 -0.1523   True
비판적 이해 생산과 공유  -0.2431    0.0 -0.2895 -0.1967   True
비판적 이해   생활활용   0.3432    0.0  0.2968  0.3896   True
 사회 참여 생산과 공유  -0.0444 0.0703 -0.0908   0.002  False
 사회 참여   생활활용   0.5419    0.0  0.4955  0.5883   True
생산과 공유   생활활용   0.5862    0.0  0.5398  0.6326   True
----------------------------------------------

In [ ]:
!pip install statsmodels scikit-posthocs
!pip install pingouin
import pingouin as pg

In [ ]:
duncan_result1 = pg.pairwise_tukey(dv='Response', between='Factor', data=long_data)
print(duncan_result1)
duncan_result2 = pg.pairwise_tukey(dv='Response', between='Age_group', data=long_data)
print(duncan_result2)

         A       B   mean(A)   mean(B)      diff        se          T  \
0     권리보호      보안  3.094459  3.164683 -0.070224  0.016281  -4.313226   
1     권리보호  비판적 이해  3.094459  3.327817 -0.233358  0.016281 -14.333007   
2     권리보호   사회 참여  3.094459  3.129106 -0.034647  0.016281  -2.128061   
3     권리보호  생산과 공유  3.094459  3.084728  0.009731  0.016281   0.597685   
4     권리보호    생활활용  3.094459  3.670974 -0.576515  0.016281 -35.409952   
5       보안  비판적 이해  3.164683  3.327817 -0.163134  0.016281 -10.019781   
6       보안   사회 참여  3.164683  3.129106  0.035577  0.016281   2.185165   
7       보안  생산과 공유  3.164683  3.084728  0.079955  0.016281   4.910911   
8       보안    생활활용  3.164683  3.670974 -0.506291  0.016281 -31.096725   
9   비판적 이해   사회 참여  3.327817  3.129106  0.198711  0.016281  12.204946   
10  비판적 이해  생산과 공유  3.327817  3.084728  0.243089  0.016281  14.930692   
11  비판적 이해    생활활용  3.327817  3.670974 -0.343157  0.016281 -21.076944   
12   사회 참여  생산과 공유  3.129106  3.084728  0.044378  0

In [ ]:
comparisons1 = duncan_result1[['A', 'B', 'diff', 'se', 'T']].copy()
df_between = len(long_data) - len(long_data['Age_group'].unique())
comparisons1['p-value'] = comparisons1.apply(lambda row: stats.t.sf(abs(row['T']), df_between) * 2, axis=1)
print(comparisons1)
comparisons2 = duncan_result2[['A', 'B', 'diff', 'se', 'T']].copy()
df_between = len(long_data) - len(long_data['Factor'].unique())
comparisons2['p-value'] = comparisons2.apply(lambda row: stats.t.sf(abs(row['T']), df_between) * 2, axis=1)
print(comparisons2)

         A       B      diff        se          T        p-value
0     권리보호      보안 -0.070224  0.016281  -4.313226   1.613668e-05
1     권리보호  비판적 이해 -0.233358  0.016281 -14.333007   1.890837e-46
2     권리보호   사회 참여 -0.034647  0.016281  -2.128061   3.333955e-02
3     권리보호  생산과 공유  0.009731  0.016281   0.597685   5.500547e-01
4     권리보호    생활활용 -0.576515  0.016281 -35.409952  1.758869e-269
5       보안  비판적 이해 -0.163134  0.016281 -10.019781   1.351027e-23
6       보안   사회 참여  0.035577  0.016281   2.185165   2.888392e-02
7       보안  생산과 공유  0.079955  0.016281   4.910911   9.109708e-07
8       보안    생활활용 -0.506291  0.016281 -31.096725  3.295267e-209
9   비판적 이해   사회 참여  0.198711  0.016281  12.204946   3.479948e-34
10  비판적 이해  생산과 공유  0.243089  0.016281  14.930692   3.063608e-50
11  비판적 이해    생활활용 -0.343157  0.016281 -21.076944   5.932974e-98
12   사회 참여  생산과 공유  0.044378  0.016281   2.725746   6.419098e-03
13   사회 참여    생활활용 -0.541868  0.016281 -33.281890  7.818783e-239
14  생산과 공유    생활활용 -0.586